# 서프라이즈 모델 파라미터 개선 및 학습 코드 

In [29]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV

def find_best_params(df, min_r, max_r):
    reader = Reader(rating_scale=(min_r, max_r))
    data = Dataset.load_from_df(df, reader=reader)
    # data_folds = DatasetAutoFolds(ratings_file='surprise_dataset_1027.csv', reader=reader)
    #trainset = data_folds_build_full_trainset()
    trainset, testset = train_test_split(data,test_size=.25,random_state=0)
    
    print('SVD 모델로 학습') 
    algo = SVD(random_state=0)
    cross_validate(algo, data, measures=['RMSE','MAE'], cv=3, verbose=True)


    # 최적화할 파라미터를 딕셔너리 형태로 지정 
    param_grid ={'n_epochs': [20,40], 'n_factors': [50]}

    gs = GridSearchCV(SVD, param_grid, measures = ['rmse','mae'],cv=3)
    gs.fit(data)

    print(gs.best_score['rmse'])
    print(gs.best_params['rmse'])
    
    return gs.best_params['rmse']
    


In [30]:
from surprise.dataset import DatasetAutoFolds

def r_predict_df(df, min_r,max_r):
    reader = Reader(rating_scale=(min_r,max_r))
    data = Dataset.load_from_df(df, reader=reader)
    
    n_epochs = 40#best_params['n_epochs']
    n_factors = 200# best_params['n_factors']   
    
    #train 과  test으로 나누지말고 전부로 모델 학습하기 
    trainset = data.build_full_trainset()
    
    #SVD 모델로 학습 
    algo = SVD(n_epochs=n_epochs ,n_factors=n_factors, random_state=0)
    algo.fit(trainset)

    print('-------------학습 완료-------------------')
    
    return algo


In [31]:
# 1. 해당 userid가 아직 평점을 매기지 않은 제품, 평점을 매긴 모든 제품 추출 (리스트 형태)
def get_unpurchased_surprise(origin_df, userid):
    # userid에 해당하는 사용자가 평점을 매긴 모든 제품
    purchased_prds = origin_df[(origin_df['CUSTNO']==userid)&(origin_df['SCORE'] > 0)]['PRD'].tolist()
    
    total_prds = origin_df['PRD'].unique()
    
    # userid에 해당하는 사용자가 한번도 구매하지 않은 모든 제품
    unpurchased_prds = [PRD for PRD in total_prds if PRD not in purchased_prds]

    # 한번도 구매하지 않은 모든 제품, 평점을 매긴 모든 제품 반출
    return unpurchased_prds,  purchased_prds

# 입력(군집별 고객 데이터, 최적의 예측값 모델, 고객번호, top 개수) -> 출력 (안사본 물품 중 top, 구매한 물품 중 top)
def recomm_prd_by_surprise(temp_df, algo,  userid, top_n):
    
    # 구매하지 않은 제품, 평점을 매긴 모든 제품 추출 각각 추출 (1. 함수 적용)
    unpurchased_prds, purchased_prds = get_unpurchased_surprise(temp_df, userid)
    
#  모델 적용
    # 1) 아직 안사본 제품
    predictions = [algo.predict(str(userid), str(PRD)) for PRD in unpurchased_prds]
    # 2) 이미 사본 제품
    predictions_bought = [algo.predict(str(userid), str(PRD)) for PRD in purchased_prds]
    
    # predictions list 객체는 predictions 객체를 원소로 갖고 있음
    # est 값으로 정렬 _ sortkey_est 함수 생성
    def sortkey_est(pred):
        return pred.est
    
# pred값을 내림 차순으로 정렬

## 1) 아직 안사본 제품
    predictions.sort(key = sortkey_est, reverse = True)
    # 상위 n개만 추출
    top_predictions = predictions[:top_n]
    
## 2) 이미 사본 제품
    predictions_bought.sort(key = sortkey_est, reverse = True)
    # 상위 n개만 추출
    top_predictions_bought = predictions_bought[:top_n]
    
# ! 최종 추출
# 상위 n 개의 제품 정보, 예측 지수
    
    # 1) 안사본 제품 
    top_prds = [pred.iid for pred in top_predictions] # 해당 제품
    top_prds_rating = [pred.est for pred in top_predictions] # 예측 지수
    
    # 2) 사본 제품
    top_prds_bought = [pred.iid for pred in top_predictions_bought] # 해당 제품
    top_prds_rating_bought = [pred.est for pred in top_predictions_bought] # 예측 지수
    
# *최종 값 형태 = 리스트 내 튜플 형태 (해당 제품, 예측 지수) ]
    top_prds_preds = [ ( id, rating) for id, rating in zip(top_prds, top_prds_rating)]
    top_prds_preds_bought = [ ( id, rating) for id, rating in zip(top_prds_bought, top_prds_rating_bought)]
    
    return top_prds_preds, top_prds_preds_bought

# 데이터프레임 불러오기
- 구매지수 R 데이터프레임 
- 고객번호와 군집 라벨 데이터프레임
- 카테고리 데이터프레임

In [32]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

surprise_df = pd.read_csv('surprise_data/surprise_dataset_1027.csv')
surprise_df.columns = ['CUSTNO', 'PRD', 'SCORE']
surprise_df.head()

,CUSTNO,PRD,SCORE
0,1,A_4대 B/D,0.0
1,1,A_5 ON THE GO,0.0
2,1,A_ACC Bloom (1F),0.0
3,1,A_ACC Bloom (3F),0.0
4,1,A_AK골프,0.0


In [33]:
# user_item_index_df
surprise_df.head()

,CUSTNO,PRD,SCORE
0,1,A_4대 B/D,0.0
1,1,A_5 ON THE GO,0.0
2,1,A_ACC Bloom (1F),0.0
3,1,A_ACC Bloom (3F),0.0
4,1,A_AK골프,0.0


In [34]:
final_cat = pd.read_csv('surprise_data/최종cat순서나열.csv', index_col=0)
final_cat.drop(columns=['대분류코드','QTY','구매금액_min','구매금액_max','구매금액_mean'], inplace=True)

In [35]:
# category_df
final_cat.head()

,제휴사,금액 타입,대분류명_y,중분류명,소분류명,UPRICE,AVG,AVGVS
0,B,저가,식품,버섯,건버섯,3017,7926,0.4
1,B,저가,식품,버섯,느타리버섯,3017,7926,0.4
2,B,저가,식품,버섯,버섯모둠,3017,7926,0.4
3,B,저가,식품,버섯,버섯선물세트,3017,7926,0.4
4,B,저가,식품,버섯,새송이버섯,3017,7926,0.4


In [41]:
# 특정 고객의 구매이력이 있는 추천리스트, 구매이력이 없는 추천리스트 딕셔너리 형태로 반환하는 함수 생성
# 입력값 (고객별 군집결과 데이터, 구매지수 데이터, 소분류명 정보_제휴사, 단가) 
# -> 반출값 : 1. 구매이력이 있는 추천리스트, 2. 구매이력이 없는 추천리스트
def dict_recommend_unrecommend(cluster_df, user_item_index_df, category_df, cluster_label): 
    
    # 반출할 빈 리스트 생성
    super_list = []
    super_list_bought = []

    # 군집 리스트
    cluster_list = list(cluster_df['cluster'].unique()) # 군집 종류
    
    
    for cluster in cluster_list[:1]:
       
        # 컴퓨터 성능 저하 문제로 임의로 cluster_label 변수를 만들어서 특정 군집 하나만 가지고 시도
        cust_index_list = list(cluster_df[cluster_df['cluster']==cluster_label].index) # 해당 군집에 있는 고객 번호 뽑기
        
        # 구매지수 데이터에서 해당하는 고객번호만 출력(새로운 데이터 생성)
        temp_df = user_item_index_df[user_item_index_df['CUSTNO'].isin(cust_index_list)] #isin(cust_index_list)]

        # 구매지수 최댓값, 최솟값 추출
        max_r = temp_df['SCORE'].max()
        min_r = temp_df['SCORE'].min()

        ## 서프라이즈 최적 파라미터  돌리기
        #best_params = find_best_params(temp_df,min_r,max_r)

         ## 서프라이즈 최적 파라미터로 모델 학습
        #algo = r_predict_df(temp_df, best_params,min_r,max_r)
        algo = r_predict_df(temp_df,min_r,max_r)

        
    # 고객별 추천 결과 받기 
# !--출력 값 (딕셔너리 형태)
        # 1) 안사본 물품
        recomm_dict = {} 
        # 2) 사본 물품
        recomm_dict_bought = {}
        
        for cust in cust_index_list:
            print("고객: ",cust)
            
            # 입력 (해당하는 군집에 있는 고객 데이터, 최적 파라미터로 모델, 고객, top) -> 추출 (안사본 물품 중 top, 구매한 물품 중 top)
            top_prd_preds, top_prd_preds_bought  = recomm_prd_by_surprise(temp_df, algo, cust, top_n=50)

            # 해당하는 고객의  입력 _(해당 제품, 구매지수) ex. {9_고객번호 : (양말_해당 제품이름, 0.5), (), () ......}
            
        # 1) 안사본 물품
            recomm_dict[cust] = [top_prd_preds[0]] # 첫번째 value 입력
            
            # 딕셔너리 차례로 담기
            for prd in top_prd_preds[1:]:
                if cust in recomm_dict:
                    recomm_dict[cust].append(prd)

        # 2) 이미 사본 제품
            recomm_dict_bought[cust] = [top_prd_preds_bought[0]]
            for prd in top_prd_preds_bought[1:]:
                if cust in recomm_dict_bought:
                    recomm_dict_bought[cust].append(prd)

# ! -- 최종 출력 값(리스트 형태) _ 군집별 고객별 제품 추천 딕셔너리
            super_list.append(recomm_dict)
            super_list_bought.append(recomm_dict_bought)
    print("super_list 완료")
    return super_list, super_list_bought

# 특정 고객의 추천 리스트 받기 

In [59]:
# 고객 
def get_recommend_cust( cust, user_item_index_df, category_df,super_list):
    
#     super_list, super_list_bought = dict_recommend_unrecommend(df, user_item_index_df, category_df)
    
    dict_custs = super_list[0]
    df = pd.DataFrame(dict_custs[cust])
    df.columns = ['ITEM','RATING']
    df['ASSO'] = df['ITEM'].apply(lambda x: x[0])
    df['ITEM'] = df['ITEM'].apply(lambda x: x[2:])

    dff = df.merge(category_df, how='left', left_on=['ASSO','ITEM'], right_on=['제휴사', '소분류명'])
    
    dff_replace = dff[['제휴사', '대분류명_y', '중분류명', '소분류명', '금액 타입', 'RATING']]
    dff_replace.columns = ['제휴사', '대분류명', '중분류명', '추천 상품', '금액 타입', '예측 구매 지수']
    dff_replace.sort_values(by=['금액 타입', '예측 구매 지수'], ascending=False)
    
    cat_grouped = dff_replace.groupby('대분류명')
    
    cat_list = list(dff_replace['대분류명'].unique())
    
    print(f' ♣ {cust} 고객님만을 위한 추천 상품은: ', '\n')
    for cat in cat_list:
        print()
        print(f' {cat} 추천 제품 ▼')
        display(cat_grouped.get_group(cat))
        
def get_recommend_cust_bought( cust, user_item_index_df, category_df, super_list_bought):
    
#     super_list, super_list_bought = dict_recommend_unrecommend(df, user_item_index_df, category_df)
    dict_custs = super_list_bought[0]
    df = pd.DataFrame(dict_custs[cust])
    df.columns = ['ITEM','RATING']
    df['ASSO'] = df['ITEM'].apply(lambda x: x[0])
    df['ITEM'] = df['ITEM'].apply(lambda x: x[2:])

    dff = df.merge(category_df, how='left', left_on=['ASSO','ITEM'], right_on=['제휴사', '소분류명'])
    
    dff_replace = dff[['제휴사', '대분류명_y', '중분류명', '소분류명', '금액 타입', 'RATING']]
    dff_replace.columns = ['제휴사', '대분류명', '중분류명', '추천 상품', '금액 타입', '예측 구매 지수']
    
    cat_grouped = dff_replace.groupby('대분류명')
    
    cat_list = list(dff_replace['대분류명'].unique())
    
    print(f' ♣ {cust} 고객님이 선호하는 상품은: ', '\n')
    # 출력값
    for cat in cat_list:
        print()
        print(f' {cat}의 추천 제품 ▼')
        display(cat_grouped.get_group(cat))
        
    
# print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<   RECOMMENDED PRODUCTS    >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>', '\n')
# get_recommend_cust(cust_df, 0,9538, final_cat)
# print('')
# print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<   PURCHSED RECOMMENDED PRODUCTS    >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>', '\n')
# get_recommend_cust_bought(cust_df, 0,9538, final_cat)


# 값 출력해보기

In [38]:
cluster_df = pd.read_csv('생성데이터/5. 추천시스템/surprise_data/cust_clustering.csv', index_col = 0)

##  고객 속성 정보 및 과거 구매 내역 같이 출력

In [49]:
import pandas as pd
import numpy as np

# 필요한 파일 저장해둔 경로입니동
## personal_all_df: second_projects/생성데이터/5. 추천시스템/surprise_data/df_사용자_아이템_고객속성정보__정리.csv
## category_df : second_projects/생성데이터/5. 추천시스템/surprise_data/category_df(중복제거).csv
## 엥겔지수 및 총구매금액 파일

# 사용자마다 경로 새로지정해줘야 함~
personal_all_df = pd.read_csv('C:/cakd7/git/class/study/git_수업용/second_projects/생성데이터/5. 추천시스템/surprise_data/df_사용자_아이템_고객속성정보_정리.csv',index_col = 0)
category_df = pd.read_csv('C:/cakd7/git/class/study/git_수업용/second_projects/생성데이터/5. 추천시스템/surprise_data/category_df(중복제거).csv',index_col = 0)


def show_personal_purchase_df(personal_all_df, category_df, asso_type, cust_number):
    # 최대 행 길이 지정
    max_len = len(personal_all_df.loc[4].dropna())
    pd.set_option('display.max_rows', max_len)

    # 개인 구매 정보 
    personal_purchase = personal_all_df.loc[cust_number].dropna()

    ## -----------속성정보 데이터프레임 생성-----------
    personal_series = personal_purchase[:4]
    personal_df = pd.DataFrame(personal_series).transpose().reset_index().rename(columns={"index": "고객 ID"})
    personal_df['엥겔지수'] = personal_df['엥겔지수'].apply(lambda x: np.round(x, 2))
    personal_df['성별'] = personal_df['성별'].apply(lambda x: '남자' if x == 'M' else '여자')
    personal_df['총 구매금액'] = personal_df['총 구매금액'].apply(lambda x: f'{x:,.0f}원')
    personal_df['6-8분기 총 구매횟수'] = np.sum(personal_purchase[4:]).astype(int)
    personal_df['6-8분기 총 구매횟수'] = personal_df['6-8분기 총 구매횟수'].apply(lambda x: f'{x:,.0f}회')


    ## -----------구매이력 데이터프레임 생성-----------
    asso_values_list = [asso_values.split('_') for asso_values in personal_purchase[5:].index]
    p_purchase_df = pd.DataFrame(asso_values_list, columns = ['제휴사', '소분류명'])

    p_purchase_df['구매횟수'] = personal_purchase.iloc[5:].values.astype(int)
    ### 중분류 대분류 컬럼 생성
    for idx in range(len(p_purchase_df)):
        asso = p_purchase_df.loc[idx,'제휴사']
        scl = p_purchase_df.loc[idx,'소분류명']

        bcl = category_df.loc[(category_df['제휴사'] == asso) & (category_df['소분류명'] == scl), '대분류명_y'].values[0]
        mcl = category_df.loc[(category_df['제휴사'] == asso) & (category_df['소분류명'] == scl), '중분류명'].values[0]
        p_purchase_df.loc[idx, '대분류명'] = bcl
        p_purchase_df.loc[idx, '중분류명'] = mcl

    ### 정렬: 구매횟수 제외하고 다 오름차순
    p_purchase_df = p_purchase_df[['제휴사','대분류명','중분류명','소분류명','구매횟수']]
    p_purchase_df = p_purchase_df.sort_values(['제휴사' ,'대분류명','중분류명','구매횟수'],
                                              ignore_index = True,
                                              ascending = [True, True, True, False])
    
    return personal_df, p_purchase_df[p_purchase_df['제휴사'] == asso_type]


# 시연 

### 입력값(특정 군집 라벨, 해당 군집 내 고객 번호) 받기 
> 시연에서는 Cluster 5 번만을 보여주고 있습니다.

In [40]:
cluster_label = 4  # Cluster 5 이지만 index는 4
super_list, super_list_bought = dict_recommend_unrecommend(cluster_df, surprise_df, final_cat, cluster_label )

[4, 1, 0, 3, 2]
해당 군집에 들어있는 첫 고객 10명:  [6, 12, 19, 22, 25, 27, 30, 31, 35, 39, 40]
-------------학습 완료-------------------
고객:  6
고객:  12
고객:  19
고객:  22
고객:  25
고객:  27
고객:  30
고객:  31
고객:  35
고객:  39
고객:  40
고객:  41
고객:  59
고객:  76
고객:  78
고객:  80
고객:  85
고객:  106
고객:  108
고객:  121
고객:  154
고객:  178
고객:  179
고객:  180
고객:  187
고객:  192
고객:  204
고객:  211
고객:  214
고객:  230
고객:  233
고객:  240
고객:  277
고객:  284
고객:  286
고객:  301
고객:  303
고객:  333
고객:  336
고객:  346
고객:  350
고객:  375
고객:  392
고객:  394
고객:  396
고객:  419
고객:  429
고객:  435
고객:  455
고객:  467
고객:  483
고객:  490
고객:  506
고객:  514
고객:  540
고객:  552
고객:  558
고객:  578
고객:  589
고객:  590
고객:  598
고객:  611
고객:  625
고객:  636
고객:  649
고객:  651
고객:  677
고객:  698
고객:  707
고객:  712
고객:  729
고객:  737
고객:  768
고객:  775
고객:  785
고객:  808
고객:  840
고객:  854
고객:  889
고객:  898
고객:  901
고객:  920
고객:  948
고객:  950
고객:  967
고객:  984
고객:  989
고객:  1015
고객:  1030
고객:  1036
고객:  1074
고객:  1097
고객:  1100
고객:  1135
고객:  1146
고객:  1158
고객:  1165
고객:  1191
고객: 

고객:  9845
고객:  9852
고객:  9853
고객:  9854
고객:  9864
고객:  9876
고객:  9896
고객:  9897
고객:  9902
고객:  9904
고객:  9906
고객:  9908
고객:  9909
고객:  9918
고객:  9921
고객:  9923
고객:  9927
고객:  9938
고객:  9951
고객:  10002
고객:  10052
고객:  10064
고객:  10091
고객:  10101
고객:  10135
고객:  10155
고객:  10173
고객:  10181
고객:  10193
고객:  10197
고객:  10227
고객:  10249
고객:  10254
고객:  10255
고객:  10269
고객:  10324
고객:  10329
고객:  10333
고객:  10341
고객:  10357
고객:  10370
고객:  10380
고객:  10389
고객:  10393
고객:  10424
고객:  10434
고객:  10486
고객:  10501
고객:  10533
고객:  10539
고객:  10571
고객:  10584
고객:  10592
고객:  10621
고객:  10660
고객:  10666
고객:  10673
고객:  10684
고객:  10756
고객:  10810
고객:  10832
고객:  10844
고객:  10866
고객:  10867
고객:  10868
고객:  10881
고객:  10884
고객:  10904
고객:  10906
고객:  10913
고객:  10920
고객:  10927
고객:  10937
고객:  10950
고객:  10968
고객:  10975
고객:  10989
고객:  11007
고객:  11014
고객:  11029
고객:  11040
고객:  11075
고객:  11079
고객:  11094
고객:  11110
고객:  11118
고객:  11131
고객:  11138
고객:  11142
고객:  11152
고객:  11164
고객:  11176
고객:  11

> Cluster 5내에 있는 고객 번호는 위와 같으며 아래에, 고객번호를 입력하여 해당 고객을 위한 추천 결과를 확인할 수 있습니다.

> Cluster 5의 특성을 바탕으로 하여 식품과 생활 고가 상품들을 앞서 보여줍니다.

In [60]:
cust_num = 392 # 고객번호

personal_df, p_purchase_df = show_personal_purchase_df(personal_all_df, category_df, 'A', cust_num)

print(f'안녕하세요, {cust_num} 고객님!  고객님의 개인정보와 과거 구매 정보는 :')
display(personal_df)
print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<   고객님의 과거 A사 구매내역  >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>', '\n')
display(p_purchase_df.sort_values(by='구매횟수', ascending=False))
print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<   RECOMMEND PRODUCTS    >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>', '\n')
get_recommend_cust( cust_num, surprise_df, final_cat, super_list)
print('')
print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<   PURCHSED RECOMMEND PRODUCTS    >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>', '\n')
get_recommend_cust_bought(cust_num, surprise_df, final_cat,super_list_bought)

안녕하세요, 392 고객님!  고객님의 개인정보와 과거 구매 정보는 :


,고객 ID,성별,연령대,엥겔지수,총 구매금액,6-8분기 총 구매횟수
0,392,여자,60세이상,29.22,"7,041,686원",822회


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<   고객님의 과거 A사 구매내역  >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 



,제휴사,대분류명,중분류명,소분류명,구매횟수
7,A,식품,가공식품,스넥(부대),256
8,A,식품,가공식품,전문베이커리,51
9,A,식품,가공식품,제과제빵,19
22,A,식품,농산물,농산가공,14
37,A,잡화,일용잡화,위생세제,9
1,A,레저취미,스포츠,몬츄라,8
10,A,식품,가공식품,포숑,7
35,A,이미용,화장품,색조 화장품,7
11,A,식품,가공식품,한식델리,5
12,A,식품,가공식품,브랜드샵,4


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<   RECOMMEND PRODUCTS    >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 

 ♣ 392 고객님만을 위한 추천 상품은:  


 식품 추천 제품 ▼


...

,제휴사,대분류명,중분류명,추천 상품,금액 타입,예측 구매 지수
19,A,레저취미,스포츠,L/C 아웃도어,중가,0.003069
24,A,레저취미,스포츠,나이키의류,중가,0.001281
41,A,레저취미,스포츠,노스페이스,중가,0.000556



 잡화의 추천 제품 ▼


,제휴사,대분류명,중분류명,추천 상품,금액 타입,예측 구매 지수
25,A,잡화,일용잡화,위생세제,저가,0.001222



 생활의 추천 제품 ▼


,제휴사,대분류명,중분류명,추천 상품,금액 타입,예측 구매 지수
43,B,생활,종량제봉투,재사용봉투,저가,0.000521
49,B,생활,야외/시즌용품,일회용품,중가,0.000435
50,B,생활,야외용품,일회용품,중가,0.000435


In [ ]:
# clust_df[clust_df ['cluster'] == 0].